In [44]:
import numpy as np
import pandas as pd

In [125]:
data = []
with open("system_input/tcp_stats.txt") as file:
    data = list(file)

In [126]:
print(data[:5])

['================================================================================\n', 'TCP Conversations\n', 'Filter:<No Filter>\n', '                                                           |       <-      | |       ->      | |     Total     |    Relative    |   Duration   |\n', '                                                           | Frames  Bytes | | Frames  Bytes | | Frames  Bytes |      Start     |              |\n']


In [127]:
data = data[5:]

In [128]:
print(data[0])

192.168.1.146:59787        <-> 192.30.252.154:80              615    910519     395     27638    1010    938157   748.442405000        57.9930



In [129]:
data = [e.split(" ") for e in data]

In [137]:
print(data)

[['192.168.1.146:59787', '', '', '', '', '', '', '', '<->', '192.30.252.154:80', '', '', '', '', '', '', '', '', '', '', '', '', '', '615', '', '', '', '910519', '', '', '', '', '395', '', '', '', '', '27638', '', '', '', '1010', '', '', '', '938157', '', '', '748.442405000', '', '', '', '', '', '', '', '57.9930\n'], ['192.168.1.146:59635', '', '', '', '', '', '', '', '<->', '23.222.154.141:443', '', '', '', '', '', '', '', '', '', '', '', '', '133', '', '', '', '143779', '', '', '', '', '151', '', '', '', '', '25693', '', '', '', '', '284', '', '', '', '169472', '', '', '', '', '0.469667000', '', '', '', '', '', '', '', '', '5.1496\n'], ['192.168.1.146:59944', '', '', '', '', '', '', '', '<->', '52.109.76.32:443', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '148', '', '', '', '217138', '', '', '', '', '115', '', '', '', '', '', '8635', '', '', '', '', '263', '', '', '', '225773', '', '2986.170638000', '', '', '', '', '', '', '', '', '2.3417\n'], ['192.168.1.146:59664', '',

In [153]:
test_element = data[0]

In [154]:
test_element = [e for e in test_element if e != '']

In [155]:
test_element = [e for e in test_element if e != '<->'] 

In [156]:
'' in test_element

False

In [157]:
print(test_element)

['192.168.1.146:59787', '192.30.252.154:80', '615', '910519', '395', '27638', '1010', '938157', '748.442405000', '57.9930\n']


In [61]:
labels = ["src_ip", "src_port", "dst_ip, dst_port", "src_frames", "src_bytes", "dst_frames", "dst_bytes", "total_frames", "total_bytes", "relative_start", "duration"]
#labels.append(data[4])

In [65]:
omit = ['', '|', '->', '<-', '|\n']
labels = [i for i in labels if i not in omit]